In [2]:
import pandas as pd
import networkx as nx
import numpy as np
np.random.seed(357823)
import random
random.seed(357823)

In [3]:
edges=pd.read_csv('edges.csv')
vertices=pd.read_csv('nodes.csv')

In [280]:
edges.head()

,source,target
0,Baron Zemo,Blackout (Marcus Daniels)
1,Baron Zemo,Vermin (comics)
2,N'astirh,S'ym
3,N'astirh,Belasco (Marvel Comics)
4,N'astirh,Madelyne Pryor


In [137]:
vertices.head()

,group,id,size
0,1,Baron Zemo,2
1,1,N'astirh,3
2,0,Silver Sable,1
3,0,Hope Summers (comics),1
4,0,Magik (Illyana Rasputin),1


In [4]:
G=nx.from_pandas_edgelist(edges,source='source',target='target')

Métricas relacionales para cada nodo.

In [31]:
from collections import Counter
degree=nx.degree(G)
triangles=nx.triangles(G)
degree_centrality= nx.degree_centrality(G)
closeness_centrality = nx.closeness_centrality(G)
betweenness_centrality = nx.betweenness_centrality(G)
eigenvector_centrality = nx.eigenvector_centrality(G)
clustering = nx.clustering(G)
tam_comp = {}
maximo=max(len(c) for c in nx.connected_components(G))
for i, component in enumerate(nx.connected_components(G)):
    for node in component:
        tam_comp[node] = len(component)
tam=Counter((len(c) for c in nx.connected_components(G)))
vertices = vertices[vertices['id'].map(lambda v: tam_comp[v] >= 10)]
print(tam)



Counter({2: 30, 3: 11, 4: 5, 5: 4, 6: 3, 7: 1, 181: 1, 11: 1})


In [32]:
degree_dict= {d[0]:d[1] for d in degree}
degreeL = [degree[v["id"]]for _,v in vertices.iterrows()]
trianglesL=[triangles[v['id']] for _,v in vertices.iterrows()]
degree_centralityL = [degree_centrality[v['id']] for _,v in vertices.iterrows()]
closeness_centralityL = [closeness_centrality[v['id']] for _,v in vertices.iterrows()]
betweenness_centralityL = [betweenness_centrality[v['id']] for _,v in vertices.iterrows()]
eigenvector_centralityL =[eigenvector_centrality[v['id']] for _,v in vertices.iterrows()]
clusteringL = [clustering[v['id']] for _,v in vertices.iterrows()]
# component_idL= [component_id[v['id']] for _,v in vertices.iterrows()]

Crear el dataframe con los datos.

In [33]:
df = pd.DataFrame({'degree':degreeL, "degree_centrality":degree_centralityL,
                  'closeness_centrality':closeness_centralityL, 'betweenness_centrality':betweenness_centralityL,
                    'eigenvector_centrality':eigenvector_centralityL,'clustering':clusteringL,'group':vertices['group']})



In [17]:
df.head()

,degree,degree_centrality,closeness_centrality,betweenness_centrality,eigenvector_centrality,clustering,group
3,1,0.002865,0.065194,0.000000,1.361871e-03,0.000000,0
6,1,0.002865,0.009551,0.000000,1.666953e-17,0.000000,1
9,6,0.017192,0.110388,0.176275,1.224383e-01,0.000000,1
14,1,0.002865,0.052361,0.000000,3.509777e-05,0.000000,2
16,3,0.008596,0.070706,0.011593,6.506930e-04,0.333333,1


In [34]:
atributos=df.iloc[:,0:-1]
objetivo=df.iloc[:,-1]

In [35]:
from sklearn.model_selection import train_test_split
(atributos_entrenamiento, atributos_prueba,
 objetivo_entrenamiento, objetivo_prueba) = train_test_split(
        # Conjuntos de datos a dividir, usando los mismos índices para ambos
        atributos, objetivo,
        # Tamaño del conjunto de prueba (20 % en este caso)
        test_size=.2,
        # Estratificación según la distribución de clases en el atributo objetivo
        stratify=objetivo,random_state=357823)

In [36]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import ColumnTransformer

normalizador = ColumnTransformer(
    [
        ('normalizador', MinMaxScaler(), list(range(0, 6)))
    ]
)

In [ ]:
from sklearn.model_selection import GridSearchCV,StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
rejilla_de_hiperparámetros = {
    # Máxima profundidad del árbol: 3, 4, 5, 6, 7, 8, 9, 10
    'arbol__max_depth': range(3, 11),
    # Mínimo número de ejemplos para poder particionar: 5, 10, 15
    'arbol__min_samples_split': range(2, 20)
}
kfold = StratifiedKFold(n_splits=10,random_state=357823,shuffle=True)
tuberia_arbol=Pipeline([('normalizador',normalizador),('arbol',DecisionTreeClassifier(random_state=357823))])
búsqueda_en_rejilla = GridSearchCV(tuberia_arbol,
                                   rejilla_de_hiperparámetros,
                                   scoring='accuracy',
                                   cv=kfold
                                   )
búsqueda_en_rejilla.fit(atributos_entrenamiento, objetivo_entrenamiento)

GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('normalizador',
                                        ColumnTransformer(transformers=[('normalizador',
                                                                         MinMaxScaler(),
                                                                         [0, 1,
                                                                          2, 3,
                                                                          4,
                                                                          5])])),
                                       ('arbol',
                                        DecisionTreeClassifier(random_state=357823))]),
             param_grid={'arbol__max_depth': range(3, 11),
                         'arbol__min_samples_split': range(2, 20)},
             scoring='accuracy')

In [352]:
búsqueda_en_rejilla.cv_results_

{'mean_fit_time': array([0.00473812, 0.00352623, 0.0032295 , 0.00150764, 0.00518565,
        0.00276318, 0.00363879, 0.00142722, 0.00482121, 0.00296917,
        0.00504513, 0.00220075, 0.0040853 , 0.00234492, 0.00231194,
        0.00510836, 0.00373545, 0.00353706, 0.00333691, 0.00434918,
        0.00396614, 0.00372291, 0.00277894, 0.00523837, 0.00514002,
        0.00604165, 0.00593388, 0.00533857, 0.00479853, 0.00534189,
        0.00389786, 0.00244741, 0.00275671, 0.00379491, 0.00600252,
        0.00462384, 0.00372336, 0.004286  , 0.00519202, 0.00504899,
        0.00430691, 0.00385289, 0.00438085, 0.00369592, 0.00303829,
        0.00457273, 0.003547  , 0.00463715, 0.00400381, 0.00269303,
        0.00406752, 0.00427001, 0.00411193, 0.0030781 , 0.00525267,
        0.0040987 , 0.00646291, 0.00480206, 0.00339425, 0.00447943,
        0.00382657, 0.00303369, 0.00400944, 0.00386491, 0.00400023,
        0.00409622, 0.0021075 , 0.00476606, 0.00441203, 0.00351932,
        0.00486841, 0.00471556,

In [42]:
print(búsqueda_en_rejilla.best_score_)

0.7329166666666667


In [43]:

from sklearn.metrics import accuracy_score
modelo_arbol=búsqueda_en_rejilla.best_estimator_
pred=modelo_arbol.predict(atributos_prueba)
print(accuracy_score(objetivo_prueba,modelo_arbol.predict(atributos_prueba)))

0.717948717948718


In [40]:
from sklearn.neighbors import KNeighborsClassifier
rejilla_de_hiperparámetros = {
    # Máxima profundidad del árbol: 3, 4, 5, 6, 7, 8, 9, 10
    'knn__metric': ["euclidean","manhattan"],
    # Mínimo número de ejemplos para poder particionar: 5, 10, 15
    'knn__n_neighbors': range(1, 11)
}
kfold = StratifiedKFold(n_splits=10,random_state=357823,shuffle=True)
tuberia_arbol=Pipeline([('normalizador',normalizador),('knn',KNeighborsClassifier())])
búsqueda_en_rejilla = GridSearchCV(tuberia_arbol,
                                   rejilla_de_hiperparámetros,
                                   scoring='accuracy',
                                   cv=kfold)
búsqueda_en_rejilla.fit(atributos_entrenamiento, objetivo_entrenamiento)

GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=357823, shuffle=True),
             estimator=Pipeline(steps=[('normalizador',
                                        ColumnTransformer(transformers=[('normalizador',
                                                                         MinMaxScaler(),
                                                                         [0, 1,
                                                                          2, 3,
                                                                          4,
                                                                          5])])),
                                       ('knn', KNeighborsClassifier())]),
             param_grid={'knn__metric': ['euclidean', 'manhattan'],
                         'knn__n_neighbors': range(1, 11)},
             scoring='accuracy')

In [26]:
búsqueda_en_rejilla.best_score_

np.float64(0.6329166666666667)

In [25]:
modelo_knn=búsqueda_en_rejilla.best_estimator_
print(accuracy_score(objetivo_prueba,modelo_knn.predict(atributos_prueba)))

0.6923076923076923


In [421]:
from sklearn.preprocessing import OneHotEncoder
from keras.layers import Normalization,Dense
from keras import Sequential, Input

In [422]:
enc=OneHotEncoder()
objetivo_entrenamiento_cod=enc.fit_transform(objetivo_entrenamiento.to_frame()).toarray()
red = Sequential()
normalizador_red=Normalization()
normalizador_red.adapt(atributos_entrenamiento.to_numpy())
red.add(Input(shape=(6,)))
red.add(normalizador_red)
red.add(Dense(10,activation="relu"))
red.add(Dense(3,activation="softmax"))
red.compile(optimizer='SGD', loss='categorical_crossentropy',
                   metrics=['accuracy'])
red.fit(atributos_entrenamiento,objetivo_entrenamiento_cod,batch_size=256,epochs=300)

Epoch 1/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 386ms/step - accuracy: 0.4861 - loss: 1.1160
Epoch 2/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 0.4861 - loss: 1.1135
Epoch 3/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - accuracy: 0.4861 - loss: 1.1110
Epoch 4/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - accuracy: 0.4861 - loss: 1.1086
Epoch 5/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - accuracy: 0.4861 - loss: 1.1062
Epoch 6/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - accuracy: 0.4861 - loss: 1.1038
Epoch 7/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step - accuracy: 0.4861 - loss: 1.1015
Epoch 8/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - accuracy: 0.4861 - loss: 1.0992
Epoch 9/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - accuracy: 0.4861 - loss: 1.0969
Epoch 10/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - accuracy: 0.4792 - loss: 1.0947
Epoch 11/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 0.4792 - loss: 1.0925
Epoch 12/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.4792 - l

In [423]:
pred=np.argmax(red.predict(atributos_prueba),axis=1)
print(accuracy_score(objetivo_prueba,pred))


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
0.5405405405405406


In [27]:
from sklearn.preprocessing import KBinsDiscretizer
discretizador = ColumnTransformer([('discretizador',
                                    KBinsDiscretizer(
    n_bins=10, 
    encode='ordinal',  
    strategy='uniform'  
),
                                   list(range(0,6)))])

In [28]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import CategoricalNB
rejilla_de_hiperparámetros = {
    # Máxima profundidad del árbol: 3, 4, 5, 6, 7, 8, 9, 10
   
    # Mínimo número de ejemplos para poder particionar: 5, 10, 15
    'naive_bayes__alpha': range(1, 11)
    
}
kfold = StratifiedKFold(n_splits=10)
tuberia_arbol=Pipeline([('discretizador',discretizador),('naive_bayes',CategoricalNB())])
búsqueda_en_rejilla = GridSearchCV(tuberia_arbol,
                                   rejilla_de_hiperparámetros,
                                   scoring='accuracy',
                                   cv=kfold)
búsqueda_en_rejilla.fit(atributos_entrenamiento, objetivo_entrenamiento)

GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=None, shuffle=False),
             estimator=Pipeline(steps=[('discretizador',
                                        ColumnTransformer(transformers=[('discretizador',
                                                                         KBinsDiscretizer(encode='ordinal',
                                                                                          n_bins=10,
                                                                                          strategy='uniform'),
                                                                         [0, 1,
                                                                          2, 3,
                                                                          4,
                                                                          5])])),
                                       ('naive_bayes', CategoricalNB())]),
             param_grid={'naive_bayes__alpha': range(1, 11)},
             scoring='accuracy')

In [29]:
búsqueda_en_rejilla.best_score_

np.float64(0.56875)

In [30]:

from sklearn.metrics import accuracy_score
modelo_arbol=búsqueda_en_rejilla.best_estimator_
pred=modelo_arbol.predict(atributos_prueba)
print(accuracy_score(objetivo_prueba,modelo_arbol.predict(atributos_prueba)))

0.5641025641025641
